In [15]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [16]:
import numpy as np
from rpy2.robjects import pandas2ri

from smoothcon import SmoothCon

pandas2ri.activate()

In [17]:
mcycle = None
%R -o mcycle library(MASS); data(mcycle)

array(['mcycle'], dtype='<U6')

In [18]:
mcycle.head()

,times,accel
1,2.4,0.0
2,2.6,-1.3
3,3.2,-2.7
4,3.6,0.0
5,4.0,-2.7


In [19]:
smooth = SmoothCon(
    spec="s(times, bs='ps', k=20, m=c(3,2))",
    data=mcycle,
    knots=None,
    absorb_cons=True,
    diagonal_penalty=True,
    pass_to_r=None,
)

In [20]:
basis = smooth.basis
penalty = smooth.penalty

In [21]:
bases = smooth.all_bases()
penalties = smooth.all_penalties()

In [22]:
times_grid = np.linspace(mcycle.times.min(), mcycle.times.max(), 100)
prediction_basis = smooth.predict({"times": times_grid})

In [23]:
prediction_bases = smooth.predict_all_bases({"times": times_grid})

In [24]:
smooth = SmoothCon(
    spec="ti(times, accel, bs='ps')",
    data=mcycle,
    knots=None,
    absorb_cons=True,
    diagonal_penalty=True,
    pass_to_r=None,
)

In [25]:
# smooth.penalty # this will raise an error

In [26]:
penalties = smooth.all_penalties()[0]
S1 = penalties[0]
S1_r = None

In [27]:
%%R -o S1_r
smooth_r <- mgcv::smoothCon(
    mgcv::ti(times, accel, bs='ps'),
    data=mcycle
)

S1_r <- smooth_r[[1]]$S[[1]]

In [28]:
np.allclose(S1, S1_r)

True